# Download and process Sentinel 1 data

This notebook downloads and processes one year of Sentinel 1 data for training and testing plots labelled in Collect Earth Online.

## John Brandt
## July 12, 2021

## Package imports, API import, source scripts

In [1]:
import datetime
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import scipy.sparse as sparse
import seaborn as sns
import yaml

from collections import Counter
from random import shuffle
from scipy.sparse.linalg import splu
from sentinelhub import WmsRequest, WcsRequest, MimeType
from sentinelhub import CRS, BBox, constants, DataSource, CustomUrlParam
from skimage.transform import resize

import reverse_geocoder as rg
import pycountry
import pycountry_convert as pc
import hickle as hkl
from shapely.geometry import Point, Polygon

with open("../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key'] 
        
%matplotlib inline
%run ../src/downloading/utils.py

In [2]:
time = ('2019-12-15', '2021-01-15')
YEAR = 2020
IMSIZE = 32

starting_days = np.cumsum([0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30])

# Bounding boxes

In [3]:
def identify_s1_layer(coords: list) -> str:
    coords = (coords[1], coords[0])
    results = rg.search(coords)
    admin1 = (results[-1]['admin1'])
    admin2 = results[-1]['admin2']
    country = results[-1]['cc']
    continent_name = pc.country_alpha2_to_continent_code(country)
    print(admin1, admin2, country, continent_name)
    if continent_name in ['AF', 'OC', 'EU']:
        layer = "SENT"
    if continent_name in ['SA']:
        if coords[0] > -7.11:
            layer = "SENT"
        else:
            layer = "SENT_DESC"
    if continent_name in ['AS']:
        if coords[0] > 23.3:
            layer = "SENT"
        else:
            layer = "SENT_DESC"
    if continent_name in ['NA']:
        layer = "SENT_DESC"
    return layer


def calc_bbox(plot_id: int, df: pd.DataFrame) -> list:
    """ Calculates the corners of a bounding box from an input
        pandas dataframe as output by Collect Earth Online

        Parameters:
         plot_id (int): plot_id of associated plot
         df (pandas.DataFrame): dataframe of associated CEO survey
    
        Returns:
         bounding_box (list): [(min(x), min(y)),
                              (max(x), max_y))]
    """
    subs = df[df['PLOT_ID'] == plot_id]
    return [(min(subs['LON']), min(subs['LAT'])),
            (max(subs['LON']), max(subs['LAT']))]


def bounding_box(points: list, expansion: int = 160) -> (tuple, 'CRS'):
    """ Calculates the corners of a bounding box with an
        input expansion in meters from a given bounding_box
        
        Subcalls:
         calculate_epsg, convertCoords

        Parameters:
         points (list): output of calc_bbox
         expansion (float): number of meters to expand or shrink the
                            points edges to be
    
        Returns:
         bl (tuple): x, y of bottom left corner with edges of expansion meters
         tr (tuple): x, y of top right corner with edges of expansion meters
    """
    bl = list(points[0])
    tr = list(points[1])
    inproj = Proj('epsg:4326')
    outproj_code = calculate_epsg(bl)
    outproj = Proj('epsg:' + str(outproj_code))
    
    bl_utm =  transform(inproj, outproj, bl[1], bl[0])
    tr_utm =  transform(inproj, outproj, tr[1], tr[0])

    distance1 = tr_utm[0] - bl_utm[0]
    distance2 = tr_utm[1] - bl_utm[1]
    expansion1 = (expansion - distance1)/2
    expansion2 = (expansion - distance2)/2
    
    bl_utm = [bl_utm[0] - expansion1, bl_utm[1] - expansion2]
    tr_utm = [tr_utm[0] + expansion1, tr_utm[1] + expansion2]
    
    zone = str(outproj_code)[3:]
    zone = zone[1:] if zone[0] == "0" else zone
    direction = 'N' if tr[1] >= 0 else 'S'
    utm_epsg = "UTM_" + zone + direction
    return (bl_utm, tr_utm), CRS[utm_epsg]

# Data download

In [6]:
def extract_dates(date_dict: dict, year: int) -> List:
    """ Transforms a SentinelHub date dictionary to a
         list of integer calendar dates
    """
    dates = []
    days_per_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
    starting_days = np.cumsum(days_per_month)
    for date in date_dict:
        if date.year == year - 1:
            dates.append(-365 + starting_days[(date.month-1)] + date.day)
        if date.year == year:
            dates.append(starting_days[(date.month-1)] + date.day)
        if date.year == year + 1:
            dates.append(365 + starting_days[(date.month-1)]+date.day)
    return dates

def identify_dates_to_download(dates: list) -> list:
    """ Identify the S1 dates to download"""
    days_per_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
    days_per_month = np.array(days_per_month)
    days_per_month = np.reshape(days_per_month, (4, 3))
    days_per_month = np.sum(days_per_month, axis = 1)

    starting_days = np.cumsum(days_per_month)

    dates = np.array(dates)
    dates_to_download = []
    for i in starting_days:
        s1_month = dates[dates > i]
        s1_month = s1_month[s1_month < (i + 30)]
        if len(s1_month) > 0:
            dates_to_download.append(s1_month[0])
    return dates_to_download


def download_sentinel_1(bbox, epsg, time = time, 
                        layer = "SENT", year = 2020, 
                        image_format = MimeType.TIFF_d16, 
                        data = DataSource.SENTINEL1_IW_ASC):
    """ Downloads all 10 and 20 meter L2A bands from sentinel-hub
        for input bbox and epsg, within time range
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): UTM EPSG associated with bbox 
         time (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
    
        Returns:
         s1 (arr): (Time, X, Y, 2) array of sentinel 1 data
         image_dates (list): number of days since time[0] for each
                              image in s1.shape[0]
    """
    try:
        print(f"The data is {data}")
        box = BBox(bbox, crs = epsg)
        image_request = WcsRequest(
                layer=layer,
                bbox=box,
                time=time,
                image_format = image_format,
                data_source= data,
                maxcc=1.0,
                resx='20m', resy='20m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                    constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=72),
            )
        
        
        s1_dates_dict = [x for x in image_request.get_dates()]
        s1_dates = extract_dates(s1_dates_dict, year)
        dates_to_download = identify_dates_to_download(s1_dates)
        
        steps_to_download = [i for i, val in enumerate(s1_dates) if val in dates_to_download]
        print(f"The following steps will be downloaded: {steps_to_download}, for {dates_to_download}")
        
        
        data_filter = steps_to_download
        if len(image_request.download_list) <= 3 or len(steps_to_download) <= 3:
            return np.empty((0,)), np.empty((0,))
        s1 = image_request.get_data(data_filter = data_filter)
        s1 = np.stack(s1)
        s1 = to_float32(s1)
        
        assert np.max(s1) <= 1.
        assert s1.shape[1] == 16.
        assert s1.shape[2] == 16.
        
        print(f"Sentinel 1 used {(2/3)*s1.shape[0] * (s1.shape[1]*s1.shape[2])/(512*512)} PU for"
              f" {s1.shape[0]} out of {len(image_request.download_list)} images")
        

        original = s1.shape
        s1 = s1.repeat(3, axis = 0)
        print(s1.shape)
        # Store it with nearest upsample, but this will be converted to bilinear at train time
        s1 = resize(s1, (s1.shape[0], 32, 32, 2), 0)
        new = s1.shape
        print(f"{original} -> {new}")
        
        image_dates = []
        for date in image_request.get_dates():
            if date.year == year - 1:
                image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
            if date.year == year:
                image_dates.append(starting_days[(date.month-1)] + date.day)
            if date.year == year + 1:
                image_dates.append(365 + starting_days[(date.month-1)]+date.day)
        image_dates = [val for idx, val in enumerate(image_dates) if idx in data_filter]
        image_dates = np.array(image_dates)
        
        s1c = np.copy(s1)
        s1c[np.where(s1c < 1.)] = 0
        n_pix_oob = np.sum(s1c, axis = (1, 2, 3))
        to_remove = np.argwhere(n_pix_oob > (32*32*2)/10)
        if len(to_remove) > 0:
            print(f'A total of {len(to_remove)} steps of {s1.shape[0]} were removed.')
            s1 = np.delete(s1, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)

        return s1, image_dates

    except Exception as e:
        logging.fatal(e, exc_info=True)

# Download function

In [8]:
def download_plots(data_location: str, output_folder: str, image_format = MimeType.TIFF_d16) -> None:
    """ Downloads sentinel-1 data for the plot IDs associated
        with an input CSV from a collect earth online survey
        
        Parameters:
         data_location (os.path)
         output_folder (os.path)
        
        Subcalls:
         calc_bbox, bounding_box
         download_sentinel_1,
         calculate_and_save_best_images
         
        Creates:
         output_folder/{plot_id}.npy
    
        Returns:
         None
    """
    df = pd.read_csv(data_location, encoding = "ISO-8859-1")
    df.columns = [x.upper() for x in df.columns]
    for column in ['IMAGERY_TITLE', 'STACKINGPROFILEDG', 'PL_PLOTID', 'IMAGERYYEARDG',
                  'IMAGERYMONTHPLANET', 'IMAGERYYEARPLANET', 'IMAGERYDATESECUREWATCH',
                  'IMAGERYENDDATESECUREWATCH', 'IMAGERYFEATUREPROFILESECUREWATCH',
                  'IMAGERYSTARTDATESECUREWATCH',
                  'IMAGERY_ATTRIBUTIONS',
                  'SAMPLE_GEOM']:
        if column in df.columns:
            df = df.drop(column, axis = 1)
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_ID'].unique())
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]

    to_download = [x for x in plot_ids if x not in existing]
    print(f"Starting download of {len(to_download)}"
          f" plots from {data_location} to {output_folder}")
    errors = []
    for i, val in enumerate(to_download):
        print(f"Downloading {i+1}/{len(to_download)}, {val}")
        location_wgs = calc_bbox(val, df = df)
        location, epsg = bounding_box(location_wgs, expansion = IMSIZE*10)
        try:
            # Identify cloud steps, download DEM, and download L2A series
            s1_layer = identify_s1_layer(location_wgs[0])
            data_source = DataSource.SENTINEL1_IW_DES if s1_layer == "SENT_DESC" else DataSource.SENTINEL1_IW_ASC
            s1, s1_dates = download_sentinel_1(location, 
                                               layer = s1_layer, 
                                               epsg = epsg,
                                               data = data_source)
            if s1.shape[0] < 2:
                s1_layer = "SENT_DESC" if s1_layer == "SENT" else "SENT"
                data_source = DataSource.SENTINEL1_IW_DES if s1_layer == "SENT_DESC" else DataSource.SENTINEL1_IW_ASC
                print(f'Switching to {s1_layer}')
                s1, s1_dates = download_sentinel_1(location, 
                                                   layer = s1_layer,
                                                   epsg = epsg,
                                                   data = data_source)
            
            s1_a = np.copy(s1)
            s1, max_distance = calculate_and_save_best_images(s1, s1_dates)

            s1_b = np.copy(s1)
            # Retain only iamgery every month
            monthly = np.empty((12, IMSIZE, IMSIZE, 2))
            index = 0
            for start, end in zip(range(0, 72 + 6, 72 // 12), #0, 72, 6
                                  range(72 // 12, 72 + 6, 72 // 12)): # 6, 72, 6
                monthly[index] = np.median(s1[start:end], axis = 0)
                index += 1

            s1 = monthly
            s1_c = np.copy(s1)
            
            assert s1.shape[1] == IMSIZE
            assert s1.shape[2] == IMSIZE
            if max_distance < 200:
                hkl.dump(s1, output_folder + str(val) + ".hkl", mode='w', compression='gzip')
                print('\n')
            else:
                print(f"Skipping {val} because max distance is {max_distance}")
            
        except Exception as e:
            print(e)
            logging.fatal(e, exc_info=True)
            errors.append(i)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

for i in (os.listdir("../data/train-csv/")):
    if "kenya" in i:
        download_plots("../data/train-csv/" + i, "../data/train-s1-radiometric-32/", image_format = MimeType.TIFF_d16)

Starting download of 177 plots from ../data/train-csv/ceo-kenya_shrubs-sample-data-2021-09-02.csv to ../data/train-s1-radiometric-32/
Loading formatted geocoded file...
Taita Taveta  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 21749
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Makueni  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 24756
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Makueni  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 40865
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16

The original max value is 24521
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Makueni  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 25148
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Makueni  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 23953
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Makueni  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 25038
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)




Garissa  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [12, 27, 41, 57], for [65, 161, 245, 341]
The original max value is 21101
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 64 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Isiolo  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [12, 27, 41, 57], for [65, 161, 245, 341]
The original max value is 35820
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 64 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Isiolo  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 27654
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Garissa  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original

The following steps will be downloaded: [5, 13, 21, 28], for [60, 156, 252, 336]
The original max value is 29582
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Mandera  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 21, 28], for [60, 156, 252, 336]
The original max value is 29390
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Mandera  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 21, 28], for [60, 156, 252, 336]
The original max value is 22864
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Mandera  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 21, 28], for [60, 156, 252, 336]
The original max value is 24413
Sentinel 1 used 0.00260416666666666

CRITICAL:root:zero-size array to reduction operation minimum which has no identity
Traceback (most recent call last):
  File "<ipython-input-8-759f489a7038>", line 60, in download_plots
    s1, max_distance = calculate_and_save_best_images(s1, s1_dates)
  File "/Users/jbrandt.terminal/Documents/GitHub/sentinel-tree-cover/src/downloading/utils.py", line 173, in calculate_and_save_best_images
    closest = np.min(abs(distances))
  File "<__array_function__ internals>", line 6, in amin
  File "/Users/jbrandt.terminal/opt/anaconda3/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 2793, in amin
    keepdims=keepdims, initial=initial, where=where)
  File "/Users/jbrandt.terminal/opt/anaconda3/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 90, in _wrapreduction
    return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
ValueError: zero-size array to reduction operation minimum which has no identity


The original max value is 65535
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 64 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)
A total of 9 steps of 12 were removed.
zero-size array to reduction operation minimum which has no identity
Wajir  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [10, 26, 41, 56], for [60, 156, 245, 336]
The original max value is 13767
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 64 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Wajir  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [10, 26, 41, 56], for [60, 156, 245, 336]
The original max value is 16451
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 64 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Wajir  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 21, 28], for [60, 156, 252, 336]
The original max value is 20085
Sentinel 1 use

The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 30837
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Kwale  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 25747
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Kwale  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 27848
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Kwale  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 32473
Sentinel 1 used 0.0026041666666666665 PU 

The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 19927
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Tana River  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 21720
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Tana River  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 18621
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Tana River  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 20448
Sentinel 1 used 0.00260416

The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 16141
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Garissa  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 11342
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Garissa  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 17337
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 32 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Garissa  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [5, 13, 20, 28], for [65, 161, 245, 341]
The original max value is 20444
Sentinel 1 used 0.00260416666666666

Taita Taveta  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 9647
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Taita Taveta  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 11604
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Tharaka District  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 154, 250, 346]
The original max value is 38938
Sentinel 1 used 0.0026041666666666665 PU for 4 out of 33 images
(12, 16, 16, 2)
(4, 16, 16, 2) -> (12, 32, 32, 2)


Tharaka District  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [7, 14, 22, 30], for [70, 